# SQL Project
You were hired by Ironhack to perform an Analytics Consulting Project entitled: competitive landscape.

Your mission is to create and populate an appropriate database with many coding schools that are our competition, as well as design an suitable queries that answer business questions of interest (to be defined by you)


**Suggested Steps in the Project:**


*   Read this notebook and understand each function. Comment the code appropriately

*   Populate the list of schools with a wider variety of schools (how are you going to get the school ID?)

* Take a look at the obtained dataframes. What dimensions do you have? what keys do you have? how could the different dataframes be connected?

* Go back to the drawing board and try to create an entity relationship diagram for tables available

* Once you have the schemas you want, you will need to:
  - create the suitable SQL queries to create the tables and populate them
  - run these queries using the appropriate Python connectors
  
* Bonus: How will this datamodel be updated in the future? Please write auxiliary functions that test the database for data quality issues. For example: how could you make sure you only include the most recent comments when you re-run the script?


# Suggested Deliverables

* 5-6 minute presentation of data model created, decision process and business analysis proposed

* exported .sql file with the final schema

* Supporting python files used to generate all logic

* High level documentation explaining tables designed and focusing on update methods

Crucial hint: check out the following tutorial:
https://www.dataquest.io/blog/sql-insert-tutorial/


In [158]:
# you must populate this dict with the schools required -> try talking to the teaching team about this


schools = {   
'ironhack' : 10828,
'udacity' : 11118,
'springboard' : 11035,
'practicum-by-yandex' : 11225,
'brainstation' : 10571,   
}

import re
import pandas as pd
from pandas.io.json import json_normalize
import requests



def get_comments_school(school):
    TAG_RE = re.compile(r'<[^>]+>')
    # defines url to make api call to data -> dynamic with school if you want to scrape competition
    url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"
    #makes get request and converts answer to json
    # url defines the page of all the information, request is made, and information is returned to data variable
    data = requests.get(url).json()
    #converts json to dataframe
    reviews =  pd.DataFrame(data['content']['reviews'])
  
    #aux function to apply regex and remove tags
    def remove_tags(x):
        return TAG_RE.sub('',x)
    reviews['review_body'] = reviews['body'].apply(remove_tags)
    reviews['school'] = school
    return reviews

In [159]:
# could you write this as a list comprehension? ;)
comments = []

#for school in schools.keys():
#   print(school)
#    comments.append(get_comments_school(school))

comments = [get_comments_school(school) for school in schools.keys()]
comments = pd.concat(comments)
comments

,id,name,anonymous,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,body,rawBody,...,queryDate,program,user,overallScore,comments,overall,curriculum,jobSupport,review_body,school
0,283970,Anonymous,True,None,2021.0,True,Innovation and strategy,Although there are some things I suggest to im...,"<span class=""truncatable""><p></p><p>Pros - Abh...",<p>Pros - Abhi is really talented as a teacher...,...,2021-12-07,Data Analytics Part-Time,{'image': None},3.7,[],4.0,4.0,3.0,Pros - Abhi is really talented as a teacher. H...,ironhack
1,283920,Pekka Tiitinen,False,None,2021.0,False,,Amazing experience,"<span class=""truncatable""><p></p><p>Very in de...",<p>Very in depth content and a hands on approa...,...,2021-12-05,Web Development Part-Time,{'image': None},4.7,[],5.0,5.0,4.0,Very in depth content and a hands on approach ...,ironhack
2,283888,Anonymous,True,None,2021.0,False,,"Overall good experience, a few things to improve","<span class=""truncatable""><p></p><p>Pluses: - ...",<p>Pluses: - a lot of things are covered durin...,...,2021-12-03,Web Development Bootcamp,{'image': None},3.7,[],4.0,4.0,3.0,Pluses: - a lot of things are covered during 9...,ironhack
3,283886,GASZTOWTT,False,None,2021.0,False,,Great !,"<span class=""truncatable""><p></p><p>I really e...",<p>I really enjoyed my whole experience at Iro...,...,2021-12-03,Web Development Bootcamp,{'image': None},3.7,[],4.0,4.0,3.0,I really enjoyed my whole experience at Ironha...,ironhack
4,283885,Marc Ramos,False,None,2021.0,False,Full stack Javascript developer,Un excellent moyen d'acquérir de bonnes bases ...,"<span class=""truncatable""><p></p><p>Participer...",<p>Participer &agrave; un bootcamp est une v&e...,...,2021-12-03,Web Development Bootcamp,{'image': None},4.3,[],5.0,4.0,4.0,Participer à un bootcamp est une véritable ave...,ironhack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,233011,Anonymous,False,None,2015.0,True,None,BrainStation's Product Management course helpe...,"<span class=""truncatable""><p>I took BrainStati...",I took BrainStation's Product Management cours...,...,2015-11-10,Product Management,{'image': None},5.0,[],5.0,NaN,NaN,I took BrainStation's Product Management cours...,brainstation
302,232993,Anonymous,False,None,2015.0,True,None,The course offers good continuing education fo...,"<span class=""truncatable""><p>The content cover...",The content covered was relevant and interesti...,...,2015-11-01,Digital Marketing,{'image': None},3.0,"[{'name': 'contact@brainstation.io', 'email': ...",3.0,NaN,NaN,The content covered was relevant and interesti...,brainstation
303,232987,Anonymous,False,None,2015.0,True,None,The Digital Marketing course is structured so ...,"<span class=""truncatable""><p>This program has ...",This program has some great insight from peopl...,...,2015-10-30,Digital Marketing,{'image': None},4.0,[],4.0,NaN,NaN,This program has some great insight from peopl...,brainstation
304,232910,Anonymous,False,None,2015.0,True,None,Built my first iOS app!,"<span class=""truncatable""><p>I am enrolled in ...",I am enrolled in the Intro to iOS Development ...,...,2015-10-24,iOS Development,{'image': None},5.0,[],5.0,NaN,NaN,I am enrolled in the Intro to iOS Development ...,brainstation


In [160]:
comments["school"].unique()

array(['ironhack', 'udacity', 'springboard', 'practicum-by-yandex',
       'brainstation'], dtype=object)

In [161]:
from pandas.io.json import json_normalize

def get_school_info(school, school_id):
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    data = requests.get(url).json()

    data.keys()

    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    badges_df = pd.DataFrame(data['content']['meritBadges'])
    
    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school
    

    locations_df['school_id'] = school_id
    courses_df['school_id'] = school_id
    badges_df['school_id'] = school_id
    school_df['school_id'] = school_id

    return locations_df, courses_df, badges_df, school_df

locations_list = []
courses_list = []
badges_list = []
schools_list = []

for school, id in schools.items():
    print(school)
    a,b,c,d = get_school_info(school,id)
    
    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)



ironhack


<ipython-input-161-7111c633f1e2>:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  locations_df = json_normalize(locations)


udacity
springboard
practicum-by-yandex
brainstation


In [162]:
locations_list

[      id               description  country.id   country.name country.abbrev  \
 0  15901           Berlin, Germany        57.0        Germany             DE   
 1  16022       Mexico City, Mexico        29.0         Mexico             MX   
 2  16086    Amsterdam, Netherlands        59.0    Netherlands             NL   
 3  16088         Sao Paulo, Brazil        42.0         Brazil             BR   
 4  16109             Paris, France        38.0         France             FR   
 5  16375  Miami, FL, United States         1.0  United States             US   
 6  16376             Madrid, Spain        12.0          Spain             ES   
 7  16377          Barcelona, Spain        12.0          Spain             ES   
 8  16709          Lisbon, Portugal        28.0       Portugal             PT   
 9  17233                    Online         NaN            NaN            NaN   
 
    city.id    city.name city.keyword  state.id state.name state.abbrev  \
 0  31156.0       Berlin       b

In [163]:
locations = pd.concat(locations_list)
locations

,id,description,country.id,country.name,country.abbrev,city.id,city.name,city.keyword,state.id,state.name,state.abbrev,state.keyword,school,school_id
0,15901,"Berlin, Germany",57.0,Germany,DE,31156.0,Berlin,berlin,NaN,NaN,NaN,NaN,ironhack,10828
1,16022,"Mexico City, Mexico",29.0,Mexico,MX,31175.0,Mexico City,mexico-city,NaN,NaN,NaN,NaN,ironhack,10828
2,16086,"Amsterdam, Netherlands",59.0,Netherlands,NL,31168.0,Amsterdam,amsterdam,NaN,NaN,NaN,NaN,ironhack,10828
3,16088,"Sao Paulo, Brazil",42.0,Brazil,BR,31121.0,Sao Paulo,sao-paulo,NaN,NaN,NaN,NaN,ironhack,10828
4,16109,"Paris, France",38.0,France,FR,31136.0,Paris,paris,NaN,NaN,NaN,NaN,ironhack,10828
5,16375,"Miami, FL, United States",1.0,United States,US,31.0,Miami,miami,11.0,Florida,FL,florida,ironhack,10828
6,16376,"Madrid, Spain",12.0,Spain,ES,31052.0,Madrid,madrid,NaN,NaN,NaN,NaN,ironhack,10828
7,16377,"Barcelona, Spain",12.0,Spain,ES,31170.0,Barcelona,barcelona,NaN,NaN,NaN,NaN,ironhack,10828
8,16709,"Lisbon, Portugal",28.0,Portugal,PT,31075.0,Lisbon,lisbon,NaN,NaN,NaN,NaN,ironhack,10828
9,17233,Online,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Online,Online,online,ironhack,10828


In [164]:
courses = pd.concat(courses_list)
courses["courses"].unique()

array(['Cyber Security Bootcamp', 'Cybersecurity Part-Time',
       'Data Analytics Bootcamp', 'Data Analytics Part-Time',
       'UX/UI Design Bootcamp', 'UX/UI Design Part-Time',
       'Web Development Bootcamp', 'Web Development Part-Time',
       'AI Programming with Python', 'Al Product Manager',
       'Android Basics', 'Android Developer', 'Artificial Intelligence',
       'Artificial Intelligence for Trading', 'Blockchain Developer',
       'Business Analytics', 'C++', 'Cloud Dev Ops Engineer',
       'Cloud Developer', 'Computer Vision', 'Data Analyst',
       'Data Engineer', 'Data Structures and Algorithms',
       'Data Visualization', 'Deep Learning',
       'Deep Reinforcement Learning', 'Digital Marketing',
       'Front End Web Developer', 'Full Stack Web Developer',
       'Introduction to Machine Learning', 'Introduction to Programming',
       'iOS Developer', 'Java Developer', 'Machine Learning Engineer',
       'Marketing Analytics', 'Natural Language Processing',

In [165]:
badges = pd.concat(badges_list)
badges.head()

,name,keyword,description,school,school_id
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828
0,Available Online,available_online,<p>School offers fully online courses</p>,udacity,11118
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,udacity,11118


In [166]:
# any data cleaning still missing here? take a look at the description
schools = pd.concat(schools_list)
schools.head()

,website,description,LogoUrl,school,school_id
0,www.ironhack.com/en,"<span class=""truncatable""><p>Ironhack is a glo...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack,10828
0,www.udacity.com/?utm_source=switchup&utm_mediu...,"<span class=""truncatable""><p>Udacity is a glob...",https://d92mrp7hetgfk.cloudfront.net/images/si...,udacity,11118
0,www.springboard.com/?utm_source=switchup&utm_m...,"<span class=""truncatable""><p>Springboard is an...",https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard,11035
0,practicum.yandex.com/?utm_source=referral&utm_...,"<span class=""truncatable""><p>Created by engine...",https://d92mrp7hetgfk.cloudfront.net/images/si...,practicum-by-yandex,11225
0,brainstation.io,"<span class=""truncatable""><p>BrainStation is t...",https://d92mrp7hetgfk.cloudfront.net/images/si...,brainstation,10571


In [167]:
cleaned_schools = locations[['school_id','school']]
cleaned_schools.columns = ['schools_id','name']
cleaned_schools = cleaned_schools.drop_duplicates(subset=['schools_id'])
cleaned_schools

,schools_id,name
0,10828,ironhack
0,11118,udacity
0,11035,springboard
0,11225,practicum-by-yandex
0,10571,brainstation


In [168]:
badges

,name,keyword,description,school,school_id
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828
0,Available Online,available_online,<p>School offers fully online courses</p>,udacity,11118
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,udacity,11118
0,Available Online,available_online,<p>School offers fully online courses</p>,springboard,11035
1,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,springboard,11035
2,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,springboard,11035
0,Available Online,available_online,<p>School offers fully online courses</p>,practicum-by-yandex,11225
1,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,practicum-by-yandex,11225


In [169]:
df2 = cleaned_schools.rename(columns={'name':'school'})
cleaned_comments = comments.merge(df2, how='inner', on='school')
display(cleaned_comments.columns)
list_of_drops = ['anonymous', 'hostProgramName', 'graduatingYear', 'tagline', 'body', 'rawBody', 'createdAt', 'user', 'comments', 'review_body','school']
cleaned_comments.drop(list_of_drops, inplace=True,axis=1)
cleaned_comments = cleaned_comments.fillna("0")
cleaned_comments['overall'] = cleaned_comments['overall'].apply(lambda x : float(x))
cleaned_comments['overallScore'] = cleaned_comments['overallScore'].apply(lambda x : float(x))
cleaned_comments['curriculum'] = cleaned_comments['curriculum'].apply(lambda x : float(x))
cleaned_comments['jobSupport'] = cleaned_comments['jobSupport'].apply(lambda x : float(x))

Index(['id', 'name', 'anonymous', 'hostProgramName', 'graduatingYear',
       'isAlumni', 'jobTitle', 'tagline', 'body', 'rawBody', 'createdAt',
       'queryDate', 'program', 'user', 'overallScore', 'comments', 'overall',
       'curriculum', 'jobSupport', 'review_body', 'school', 'schools_id'],
      dtype='object')

In [170]:
cleaned_comments

,id,name,isAlumni,jobTitle,queryDate,program,overallScore,overall,curriculum,jobSupport,schools_id
0,283970,Anonymous,True,Innovation and strategy,2021-12-07,Data Analytics Part-Time,3.7,4.0,4.0,3.0,10828
1,283920,Pekka Tiitinen,False,,2021-12-05,Web Development Part-Time,4.7,5.0,5.0,4.0,10828
2,283888,Anonymous,False,,2021-12-03,Web Development Bootcamp,3.7,4.0,4.0,3.0,10828
3,283886,GASZTOWTT,False,,2021-12-03,Web Development Bootcamp,3.7,4.0,4.0,3.0,10828
4,283885,Marc Ramos,False,Full stack Javascript developer,2021-12-03,Web Development Bootcamp,4.3,5.0,4.0,4.0,10828
...,...,...,...,...,...,...,...,...,...,...,...
3979,233011,Anonymous,True,0,2015-11-10,Product Management,5.0,5.0,0.0,0.0,10571
3980,232993,Anonymous,True,0,2015-11-01,Digital Marketing,3.0,3.0,0.0,0.0,10571
3981,232987,Anonymous,True,0,2015-10-30,Digital Marketing,4.0,4.0,0.0,0.0,10571
3982,232910,Anonymous,True,0,2015-10-24,iOS Development,5.0,5.0,0.0,0.0,10571


In [171]:
cleaned_locations = locations.copy()
list_to_drop = ['description','country.id','country.abbrev','city.id','city.keyword','state.id','state.name','state.abbrev','state.keyword']
cleaned_locations.drop(list_to_drop,inplace=True,axis=1,)
cleaned_locations.rename(columns = {'id':'location_id','country.name':'country','city.name':'city'}, inplace = True)
cleaned_locations = cleaned_locations.fillna(value = "Online")

In [172]:
badges_raw = pd.concat(badges_list)
badges_raw = badges_raw.drop_duplicates(subset=['name'])
badges_raw

,name,keyword,description,school,school_id
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack,10828
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack,10828
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack,10828
2,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,springboard,11035


In [173]:

def badges_m(row):
    if row == 'Available Online':
        return 1
    elif row == 'Verified Outcomes':
        return 2
    elif row == 'Flexible Classes':
        return 3
    elif row == 'Job Guarantee':
        return 4

In [174]:
badges_raw.insert(0,'badges_id',(badges_raw['name'].apply(badges_m)))


In [175]:
badges = badges_raw


In [176]:
badges.columns = ['badges_id','name','keyword','description','school','school_id']


In [177]:
clean_badges = badges[['badges_id','name']]


In [178]:
clean_badges

,badges_id,name
0,1,Available Online
1,2,Verified Outcomes
2,3,Flexible Classes
2,4,Job Guarantee


In [179]:
subdf = pd.concat(badges_list)
badges_for_schools = subdf[['name','school_id']]
badges_for_schools = badges_for_schools.merge(clean_badges,how='inner',on='name')
badges_for_schools.insert(0,'school_badges_id',(range(1,13)))
badges_for_schools = badges_for_schools[['school_badges_id','school_id','badges_id']]


In [180]:
badges_for_schools

,school_badges_id,school_id,badges_id
0,1,10828,1
1,2,11118,1
2,3,11035,1
3,4,11225,1
4,5,10571,1
5,6,10828,2
6,7,10828,3
7,8,11118,3
8,9,11035,3
9,10,10571,3


In [181]:
display(badges_for_schools)
display(cleaned_comments)
display(cleaned_schools)
display(cleaned_locations)
display(courses)

,school_badges_id,school_id,badges_id
0,1,10828,1
1,2,11118,1
2,3,11035,1
3,4,11225,1
4,5,10571,1
5,6,10828,2
6,7,10828,3
7,8,11118,3
8,9,11035,3
9,10,10571,3


,id,name,isAlumni,jobTitle,queryDate,program,overallScore,overall,curriculum,jobSupport,schools_id
0,283970,Anonymous,True,Innovation and strategy,2021-12-07,Data Analytics Part-Time,3.7,4.0,4.0,3.0,10828
1,283920,Pekka Tiitinen,False,,2021-12-05,Web Development Part-Time,4.7,5.0,5.0,4.0,10828
2,283888,Anonymous,False,,2021-12-03,Web Development Bootcamp,3.7,4.0,4.0,3.0,10828
3,283886,GASZTOWTT,False,,2021-12-03,Web Development Bootcamp,3.7,4.0,4.0,3.0,10828
4,283885,Marc Ramos,False,Full stack Javascript developer,2021-12-03,Web Development Bootcamp,4.3,5.0,4.0,4.0,10828
...,...,...,...,...,...,...,...,...,...,...,...
3979,233011,Anonymous,True,0,2015-11-10,Product Management,5.0,5.0,0.0,0.0,10571
3980,232993,Anonymous,True,0,2015-11-01,Digital Marketing,3.0,3.0,0.0,0.0,10571
3981,232987,Anonymous,True,0,2015-10-30,Digital Marketing,4.0,4.0,0.0,0.0,10571
3982,232910,Anonymous,True,0,2015-10-24,iOS Development,5.0,5.0,0.0,0.0,10571


,schools_id,name
0,10828,ironhack
0,11118,udacity
0,11035,springboard
0,11225,practicum-by-yandex
0,10571,brainstation


,location_id,country,city,school,school_id
0,15901,Germany,Berlin,ironhack,10828
1,16022,Mexico,Mexico City,ironhack,10828
2,16086,Netherlands,Amsterdam,ironhack,10828
3,16088,Brazil,Sao Paulo,ironhack,10828
4,16109,France,Paris,ironhack,10828
5,16375,United States,Miami,ironhack,10828
6,16376,Spain,Madrid,ironhack,10828
7,16377,Spain,Barcelona,ironhack,10828
8,16709,Portugal,Lisbon,ironhack,10828
9,17233,Online,Online,ironhack,10828


,courses,school,school_id
0,Cyber Security Bootcamp,ironhack,10828
1,Cybersecurity Part-Time,ironhack,10828
2,Data Analytics Bootcamp,ironhack,10828
3,Data Analytics Part-Time,ironhack,10828
4,UX/UI Design Bootcamp,ironhack,10828
...,...,...,...
14,Web Development Certificate Course,brainstation,10571
15,Data Science Diploma Program,brainstation,10571
16,Digital Marketing Diploma Program,brainstation,10571
17,User Experience Design Diploma Program,brainstation,10571


In [182]:
lst = list()
for i in courses['courses']:
    if i.find('Data') != -1:
        lst.append('Data')
    else:
        lst.append(i)
courses['courses'] = pd.Series(lst)
courses

,courses,school,school_id
0,Cyber Security Bootcamp,ironhack,10828
1,Cybersecurity Part-Time,ironhack,10828
2,Data,ironhack,10828
3,Data,ironhack,10828
4,UX/UI Design Bootcamp,ironhack,10828
...,...,...,...
14,Blockchain Developer,brainstation,10571
15,Business Analytics,brainstation,10571
16,C++,brainstation,10571
17,Cloud Dev Ops Engineer,brainstation,10571


In [183]:
word_list=['Data', 'Analytics']
def change(row):
    for element in word_list:
        if element in row:
            return 'DA'
        else:
            return row





cleaned_comments['program']=cleaned_comments['program'].apply(change)


In [184]:
cleaned_comments['jobTitle'].unique()

array(['Innovation and strategy', '', 'Full stack Javascript developer',
       'UX/UI Designer', 'UX UI Designer', 'Full-Stack developer',
       'Web developer', 'Product Designer', 'Full Stack Developer',
       'full stack developer', 'UXUI Designer', 'Frontend Developer',
       'Product designer', 'ux ui designer', 'web dev fullstack',
       'Junior Data Analyst', 'Data Analyst', 'Web Developer',
       'Junior Web Dev', 'UX/UI designer', 'Growth engineer/marketer',
       'Senior Associate', 'Data Analytics', 'Software Engineer',
       'Ux ui designer', 'ux/ui product designer',
       'Fullstack web developer', 'Full stack developer',
       'Development Operations Engineer', 'Web Dev', 'UX Designer',
       'Full-Stack Web Developer', 'Student',
       'Head of Sales / Fullstack developer',
       'Power BI Junior Consultant', 'UX/UI Design',
       'Full Stack Web Developer', 'unemployed', 'frontend developer',
       'UX designer', 'Web Developer junior', 'web dev', 'web d

In [185]:
from sqlalchemy import create_engine

In [186]:
host="localhost"
port=3306
dbname="project"
user="root"
password="Popmaster17+"

In [187]:
cnx = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{dbname}").connect()

In [188]:
courses.to_sql("courses", con = cnx, if_exists="replace")

In [189]:
cleaned_comments.to_sql("comments", con = cnx, if_exists="replace")

In [190]:
cleaned_locations.to_sql("locations", con = cnx, if_exists="replace")

In [191]:
cleaned_schools.to_sql("schools", con = cnx, if_exists="replace")

In [192]:
badges_for_schools.to_sql("badges_schools", con = cnx, if_exists="replace")

In [38]:
# Q1 
# Que aspetos a Ironhack pode melhorar relativamente às concorrentes com base nos indicadores (overall, jobsupport, curriculum) 
# – corr dos indicadores com o overallscore média de cada indicador mostra 
# que a Ironhack perde nos indicadores ‘overall’ e ‘curriculum’, 
# entretanto é a escola mais forte em ‘jubsupport’Empregabilidade do curso – como o jobsupport mudou ao longo tempo

#Q2 
#Empregabilidade do curso – como o jobsupport mudou ao longo tempo

#Q3
# -Como as pessoas da área avaliam o curso (não alumni e tenha o job title)

In [201]:
query1 = pd.read_sql_query("""select AVG(overallScore), schools_id
from (select * from project.comments
   where program = "DA") sub
   group by schools_id
   order by AVG(overallScore) asc;
""", cnx)



In [202]:
query1

,AVG(overallScore),schools_id
0,4.486957,10571
1,4.511905,10828
2,4.560706,11035
3,4.614894,11118
4,4.779545,11225


In [ ]:
distribution_ironhack = pd.read_sql_query("""select overallScore, school from (select * from project.comments where program = "DA") sub
where schools_id = 10828;""", cnx)

distribution_udacity = pd.read_sql_query("""select overallScore, school from (select * from project.comments where program = "DA") sub
where schools_id = 11118;""", cnx)

distribution_springboard = pd.read_sql_query("""select overallScore, school from (select * from project.comments where program = "DA") sub
where schools_id = 11035;""", cnx)

distribution_practicum = pd.read_sql_query("""select overallScore, school from (select * from project.comments where program = "DA") sub
where schools_id = 11225;""", cnx)

distribution_brainstation = pd.read_sql_query("""select overallScore, school from (select * from project.comments where program = "DA") sub
where schools_id = 10571;""", cnx)

In [ ]:
scores = ["E","D","C","B","A","A+"]

distribution_ironhack["Ranking"] = pd.cut(distribution_ironhack["overallScore"], bins=[0,0.9,1.9,2.9,3.9,4.6,5], labels= scores)
(distribution_ironhack["Ranking"].groupby(distribution_ironhack["Ranking"]).count() / distribution_ironhack["Ranking"].count()) * 100

In [ ]:
distribution_udacity["Ranking"] = pd.cut(distribution_udacity["overallScore"], bins=[0,0.9,1.9,2.9,3.9,4.6,5], labels= scores)
(distribution_udacity["Ranking"].groupby(distribution_udacity["Ranking"]).count() / distribution_udacity["Ranking"].count()) * 100

In [ ]:
distribution_springboard["Ranking"] = pd.cut(distribution_springboard["overallScore"], bins=[0,0.9,1.9,2.9,3.9,4.6,5], labels= scores)
(distribution_springboard["Ranking"].groupby(distribution_springboard["Ranking"]).count() / distribution_springboard["Ranking"].count()) * 100

In [ ]:
distribution_brainstation["Ranking"] = pd.cut(distribution_brainstation["overallScore"], bins=[0,0.9,1.9,2.9,3.9,4.6,5], labels= scores)
(distribution_brainstation["Ranking"].groupby(distribution_brainstation["Ranking"]).count() / distribution_brainstation["Ranking"].count()) * 100

In [ ]:
distribution_practicum["Ranking"] = pd.cut(distribution_practicum["overallScore"], bins=[0,0.9,1.9,2.9,3.9,4.6,5], labels= scores)
(distribution_practicum["Ranking"].groupby(distribution_practicum["Ranking"]).count() / distribution_practicum["Ranking"].count()) * 100

In [205]:
query2 = pd.read_sql_query("""select AVG(overall), schools_id
from (select * from project.comments
   where program = "DA") sub
   group by schools_id
   order by AVG(overall) asc;
""", cnx)


In [206]:
query2

,AVG(overall),schools_id
0,4.595238,10828
1,4.671082,11035
2,4.717391,10571
3,4.723404,11118
4,4.886364,11225


In [ ]:
scores2 = ["F","D","C","B","A"]

In [ ]:
distribution_ironhack4 = pd.read_sql_query("""select overall, school from (select * from project.comments where program = "DA") sub
where schools_id = 10828;""", cnx)

distribution_udacity4 = pd.read_sql_query("""select overall, school from (select * from project.comments where program = "DA") sub
where schools_id = 11118;""", cnx)

distribution_springboard4 = pd.read_sql_query("""select overall, school from (select * from project.comments where program = "DA") sub
where schools_id = 11035;""", cnx)

distribution_practicum4 = pd.read_sql_query("""select overall, school from (select * from project.comments where program = "DA") sub
where schools_id = 11225;""", cnx)

distribution_brainstation4 = pd.read_sql_query("""select overall, school from (select * from project.comments where program = "DA") sub
where schools_id = 10571;""", cnx)

In [ ]:
distribution_ironhack4["Ranking overall"] = pd.cut(distribution_ironhack4["overall"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_ironhack4["Ranking overall"].groupby(distribution_ironhack4["Ranking overall"]).count() / distribution_ironhack4["Ranking overall"].count()) * 100

In [ ]:
distribution_udacity4["Ranking overall"] = pd.cut(distribution_udacity4["overall"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_udacity4["Ranking overall"].groupby(distribution_udacity4["Ranking overall"]).count() / distribution_udacity4["Ranking overall"].count()) * 100

In [ ]:
distribution_springboard4["Ranking overall"] = pd.cut(distribution_springboard4["overall"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_springboard4["Ranking overall"].groupby(distribution_springboard4["Ranking overall"]).count() / distribution_springboard4["Ranking overall"].count()) * 100

In [ ]:
distribution_brainstation4["Ranking overall"] = pd.cut(distribution_brainstation4["overall"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_brainstation4["Ranking overall"].groupby(distribution_brainstation4["Ranking overall"]).count() / distribution_brainstation4["Ranking overall"].count()) * 100

In [ ]:
distribution_practicum4["Ranking overall"] = pd.cut(distribution_practicum4["overall"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_practicum4["Ranking overall"].groupby(distribution_practicum4["Ranking overall"]).count() / distribution_practicum4["Ranking overall"].count()) * 100

In [207]:
query3 = pd.read_sql_query("""select AVG(curriculum), schools_id
from (select * from project.comments
   where program = "DA") sub
   group by schools_id
   order by AVG(curriculum) asc;
""", cnx)


In [ ]:
query3

In [ ]:
distribution_ironhack2 = pd.read_sql_query("""select curriculum, school from (select * from project.comments where program = "DA") sub
where schools_id = 10828;""", cnx)

distribution_udacity2 = pd.read_sql_query("""select curriculum, school from (select * from project.comments where program = "DA") sub
where schools_id = 11118;""", cnx)

distribution_springboard2 = pd.read_sql_query("""select curriculum, school from (select * from project.comments where program = "DA") sub
where schools_id = 11035;""", cnx)

distribution_practicum2 = pd.read_sql_query("""select curriculum, school from (select * from project.comments where program = "DA") sub
where schools_id = 11225;""", cnx)

distribution_brainstation2 = pd.read_sql_query("""select curriculum, school from (select * from project.comments where program = "DA") sub
where schools_id = 10571;""", cnx)

In [ ]:
distribution_ironhack2["Ranking Curriculum"] = pd.cut(distribution_ironhack2["curriculum"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_ironhack2["Ranking Curriculum"].groupby(distribution_ironhack2["Ranking Curriculum"]).count() / distribution_ironhack2["Ranking Curriculum"].count()) * 100

In [ ]:
distribution_udacity2["Ranking Curriculum"] = pd.cut(distribution_udacity2["curriculum"], bins=[0.9,1.9,2.9,3.9,4.6,5], labels= scores2)
(distribution_udacity2["Ranking Curriculum"].groupby(distribution_udacity2["Ranking Curriculum"]).count() / distribution_udacity2["Ranking Curriculum"].count()) * 100

In [ ]:
distribution_springboard2["Ranking curriculum"] = pd.cut(distribution_springboard2["curriculum"], bins=[0.9,1.9,2.9,3.9,4.6,5], labels= scores2)
(distribution_springboard2["Ranking curriculum"].groupby(distribution_springboard2["Ranking curriculum"]).count() / distribution_springboard2["Ranking curriculum"].count()) * 100

In [ ]:
distribution_brainstation2["Ranking curriculum"] = pd.cut(distribution_brainstation2["curriculum"], bins=[0.9,1.9,2.9,3.9,4.6,5], labels= scores2)
(distribution_brainstation2["Ranking curriculum"].groupby(distribution_brainstation2["Ranking curriculum"]).count() / distribution_brainstation2["Ranking curriculum"].count()) * 100

In [ ]:
distribution_practicum2["Ranking curriculum"] = pd.cut(distribution_practicum2["curriculum"], bins=[0.9,1.9,2.9,3.9,4.6,5], labels= scores2)
(distribution_practicum2["Ranking curriculum"].groupby(distribution_practicum2["Ranking curriculum"]).count() / distribution_practicum2["Ranking curriculum"].count()) * 100

,AVG(curriculum),schools_id
0,4.500000,10828
1,4.505519,11035
2,4.630435,10571
3,4.765957,11118
4,4.795455,11225


In [209]:
query4 = pd.read_sql_query("""select AVG(jobSupport), schools_id
from (select * from project.comments
   where program = "DA") sub
   group by schools_id
   order by AVG(jobSupport) asc;
""", cnx)

In [210]:
query4

,AVG(jobSupport),schools_id
0,3.586957,10571
1,4.125828,11035
2,4.148936,11118
3,4.181818,11225
4,4.428571,10828


In [ ]:
distribution_ironhack3 = pd.read_sql_query("""select jobSupport, school from (select * from project.comments where program = "DA") sub
where schools_id = 10828;""", cnx)

distribution_udacity3 = pd.read_sql_query("""select jobSupport, school from (select * from project.comments where program = "DA") sub
where schools_id = 11118;""", cnx)

distribution_springboard3 = pd.read_sql_query("""select jobSupport, school from (select * from project.comments where program = "DA") sub
where schools_id = 11035;""", cnx)

distribution_practicum3 = pd.read_sql_query("""select jobSupport, school from (select * from project.comments where program = "DA") sub
where schools_id = 11225;""", cnx)

distribution_brainstation3 = pd.read_sql_query("""select jobSupport, school from (select * from project.comments where program = "DA") sub
where schools_id = 10571;""", cnx)

In [ ]:
distribution_ironhack3["Ranking jobSupport"] = pd.cut(distribution_ironhack3["jobSupport"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_ironhack3["Ranking jobSupport"].groupby(distribution_ironhack3["Ranking jobSupport"]).count() / distribution_ironhack3["Ranking jobSupport"].count()) * 100

In [ ]:
distribution_udacity3["Ranking jobSupport"] = pd.cut(distribution_udacity3["jobSupport"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_udacity3["Ranking jobSupport"].groupby(distribution_udacity3["Ranking jobSupport"]).count() / distribution_udacity3["Ranking jobSupport"].count()) * 100

In [ ]:
distribution_springboard3["Ranking jobSupport"] = pd.cut(distribution_springboard3["jobSupport"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_springboard3["Ranking jobSupport"].groupby(distribution_springboard3["Ranking jobSupport"]).count() / distribution_springboard3["Ranking jobSupport"].count()) * 100

In [ ]:
distribution_brainstation3["Ranking jobSupport"] = pd.cut(distribution_brainstation3["jobSupport"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_brainstation3["Ranking jobSupport"].groupby(distribution_brainstation3["Ranking jobSupport"]).count() / distribution_brainstation3["Ranking jobSupport"].count()) * 100

In [ ]:
distribution_practicum3["Ranking jobSupport"] = pd.cut(distribution_practicum3["jobSupport"], bins=[0,1,2,3,4,5], labels= scores2)
(distribution_practicum3["Ranking jobSupport"].groupby(distribution_practicum3["Ranking jobSupport"]).count() / distribution_practicum3["Ranking jobSupport"].count()) * 100

In [ ]:
query_webdev1 = pd.read_sql_query("""select AVG(overallScore) as "Overall Score", school as "School", count(overallScore)
as "Total number of surveys"
from (select * from project.comments
   where program LIKE "%Web%") sub
   group by schools_id
   order by AVG(overallScore) asc;
""", cnx)

In [ ]:
query_webdev1

In [ ]:
query_webdev2 = pd.read_sql_query("""select AVG(overall) as "Overall", school as "School", count(overall)
as "Total number of surveys"
from (select * from project.comments
   where program LIKE "%Web%") sub
   group by schools_id
   order by AVG(overall) asc;
""", cnx)

In [ ]:
query_webdev2

In [ ]:
query_webdev3 = pd.read_sql_query("""select AVG(curriculum) as "Curriculum", school as "School", count(curriculum)
as "Total number of surveys"
from (select * from project.comments
   where program LIKE "%Web%") sub
   group by schools_id
   order by AVG(curriculum) asc;
""", cnx)

In [ ]:
query_webdev3

In [ ]:
query_webdev4 = pd.read_sql_query("""select AVG(jobSupport) as "Employability", school as "School", count(jobSupport)
as "Total number of surveys"
from (select * from project.comments
   where program LIKE "%Web%") sub
   group by schools_id
   order by AVG(jobSupport) asc;
""", cnx)

In [ ]:
query_webdev4

In [ ]:
#Q2 # Count total surveys vs count(jobsupport)

query_q2 = pd.read_sql_query("""select AVG(jobSupport), graduatingYear, count(jobSupport)
from project.comments
where schools_id = 10828
group by graduatingYear
order by graduatingYear asc;""", cnx)

In [ ]:
query_q2

In [ ]:
'Junior Data Analyst', 'Data Analyst' 'Data Analytics' 'Data  analyst' 'Data Analyst Jr' 'Data Analytics Student' 
'Junior data analyst' 'Finance Analyst' 'Data analyst'  'Engenheira de dados' 'Data Engineer'
'Data Science and Advanced Analytics Manager' 'Web Content Analyst' 'Digital Analyst' 'Programmer Analyst'
'IB Analyst' 'Digital Analyst' 'electrical engineer &amp;data analyst' 'Business Analytics Nanodegrees'

In [ ]:
#Q3 
query_q3 = pd.read_sql_query("""select AVG(overallScore), schools_id, count(overallScore)
from (select * from project.comments where isAlumni = 0) sub
where jobTitle like "%Data%"
	or jobTitle like "%data%"
	or jobTitle like "%Analyst%"
	or jobTitle like "%analyst%"
    or jobTitle like "%analytics%"
	or jobTitle like "%Analytics%"
	or jobTitle like "%dados%"
group by schools_id
order by AVG(overallScore) asc;""", cnx)

In [ ]:
query_q3
